In [ ]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [ ]:
#拼训练数据
csv_folder = r'.\data\out'
output_folder = r'.\data'
csv_list = os.listdir(csv_folder)
output_path = output_folder + "\\" + 'final_data.csv'

call_X = []
call_Y = []
put_X = []
put_Y = []
# 加一个混合
# 做一个对比

for csv_name in csv_list:
    if int(csv_name.split('.')[0]) % 100 == 0:
        print(csv_name)
    perdata = []
    '''numbers -= 1
    if numbers == 0:
        break'''
    csv_path = csv_folder +"\\"+csv_name
    data = pd.read_csv(csv_path, encoding = 'GBK', index_col= False)
    if len(data) < 11:
        continue

    datatype = data['call_or_put'][0]
    ll = 0

    for i in range(len(data)-1):
        onedata = list(data.iloc[i,3:])
        perdata.append(onedata)
        ll += 1
        if ll == 10:
            if datatype == 'call':
                call_X.append(perdata)
                call_Y.append(data['iv'][i+1])
            else:
                put_X.append(perdata)
                put_Y.append(data['iv'][i+1])
            ll -= 1
            perdata = perdata[1:]


In [ ]:
call_X = np.array(call_X)
call_Y = np.array(call_Y)

put_X = np.array(put_X)
put_Y = np.array(put_Y)

# 数据处理
for i in range(len(call_X)):
    if call_Y[i] <= 0:
        call_Y[i] = -call_Y[i]
        print(i, call_Y[i])
    for j in range(10):
        for k in range(1, 9):
            if call_X[i][j][k] <= 0:
                call_X[i][j][k] = -call_X[i][j][k]
                print(i,j,k,call_X[i][j][k])

for i in range(len(put_X)):
    if put_Y[i] <= 0:
        put_Y[i] = -put_Y[i]
        print(i, put_Y[i])
    for j in range(10):
        for k in range(1, 9):
            if put_X[i][j][k] <= 0:
                put_X[i][j][k] = -put_X[i][j][k]
                print(i,j,k,put_X[i][j][k])
# 正则化


In [ ]:
from sklearn.model_selection import train_test_split

call_train_X, call_val_X, call_train_Y, call_val_Y = train_test_split(call_X,
                                                                      call_Y,
                                                                      test_size=0.2,
                                                                      random_state=19011)

put_train_X, put_val_X, put_train_Y, put_val_Y = train_test_split(put_X,
                                                                  put_Y,
                                                                  test_size=0.2,
                                                                  random_state=19011)

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D , Flatten
from keras.optimizers import Adam
import tensorflow as tf

def Cov_model():
    model = Sequential()
    model.add(Conv1D(32, 2, padding='valid', input_shape=(10, 9), activation="selu"))
    model.add(Conv1D(64, 2, padding='valid', activation="selu"))
    model.add(Conv1D(128, 2, padding='valid', activation="selu"))
    model.add(Conv1D(256, 2, padding='valid', activation="selu"))
    #model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1))
    model.compile(loss='mae', optimizer='adam')
    opt = Adam(lr=0.001, clipnorm=1)
    model.compile(optimizer=opt, loss=tf.keras.losses.mean_squared_error,
                  metrics=['mae'])
    model.summary()
    return model

# fit network
cov_model = Cov_model()
history = cov_model.fit(call_train_X, call_train_Y,
                    epochs=50, batch_size=1,
                    validation_data=(call_val_X, call_val_Y),
                    verbose=1,
                    shuffle=False)

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 9, 32)             608       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 64)             4160      
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 7, 128)            16512     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 6, 256)            65792     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1536)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1537      
Total params: 88,609
Trainable params: 88,609
Non-trainable params: 0
_________________________________________________________________


NameError: name 'call_train_X' is not defined

In [ ]:
import cv2
import numpy as np


# 固定尺寸
def resizeImg(image, height=900):
    h, w = image.shape[:2]
    pro = height / h
    size = (int(w * pro), int(height))
    img = cv2.resize(image, size)
    return img


# 边缘检测
def getCanny(image):
    # 高斯模糊
    binary = cv2.GaussianBlur(image, (3, 3), 2, 2)
    # 边缘检测
    binary = cv2.Canny(binary, 60, 240, apertureSize=3)
    # 膨胀操作，尽量使边缘闭合
    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.dilate(binary, kernel, iterations=1)
    return binary


# 求出面积最大的轮廓
def findMaxContour(image):
    # 寻找边缘
    _, contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    # 计算面积
    max_area = 0.0
    max_contour = []
    for contour in contours:
        currentArea = cv2.contourArea(contour)
        if currentArea > max_area:
            max_area = currentArea
            max_contour = contour
    return max_contour, max_area


# 多边形拟合凸包的四个顶点
def getBoxPoint(contour):
    # 多边形拟合凸包
    hull = cv2.convexHull(contour)
    epsilon = 0.02 * cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(hull, epsilon, True)
    approx = approx.reshape((len(approx), 2))
    return approx


# 适配原四边形点集
def adaPoint(box, pro):
    box_pro = box
    if pro != 1.0:
        box_pro = box/pro
    box_pro = np.trunc(box_pro)
    return box_pro


# 四边形顶点排序，[top-left, top-right, bottom-right, bottom-left]
def orderPoints(pts):
    rect = np.zeros((4, 2), dtype="float32")
    s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect


# 计算长宽
def pointDistance(a, b):
    return int(np.sqrt(np.sum(np.square(a - b))))


# 透视变换
def warpImage(image, box):
    w, h = pointDistance(box[0], box[1]), \
           pointDistance(box[1], box[2])
    dst_rect = np.array([[0, 0],
                         [w - 1, 0],
                         [w - 1, h - 1],
                         [0, h - 1]], dtype='float32')
    M = cv2.getPerspectiveTransform(box, dst_rect)
    warped = cv2.warpPerspective(image, M, (w, h))
    return warped


if __name__ == '__main__':
    path = r'C:\Users\wzg\Desktop\test.jpg'
    outpath = r'C:\Users\wzg\Desktop\result.jpg'
    image = cv2.imread(path)
    ratio = 900 / image.shape[0]
    img = resizeImg(image)
    binary_img = getCanny(img)
    max_contour, max_area = findMaxContour(binary_img)
    boxes = getBoxPoint(max_contour)
    boxes = adaPoint(boxes, ratio)
    boxes = orderPoints(boxes)
    # 透视变化
    warped = warpImage(image, boxes)
    cv2.imshow('warpImage', warped)
    cv2.waitKey(0)